# Personality Atlas — Quick Start

**Reproduce the atlas results in under 5 minutes.** No API keys required.

> Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST. Under review.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Wildertrek/survey/blob/main/notebooks/atlas_quick_start.ipynb)

---

## 1. Setup (~30 seconds)

In [ ]:
# Clone the atlas repository (skip if already cloned)
import os
if not os.path.exists("atlas"):
    !git clone --depth 1 https://github.com/Wildertrek/survey.git atlas
else:
    print("Atlas already cloned — skipping.")

In [ ]:
# Install dependencies (uses Colab's pre-installed sklearn — no version conflicts)
!pip install -q faiss-cpu

# Suppress sklearn version warning (models trained with 1.5.0, works fine with newer versions)
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

## 2. Load and Predict — Any of 44 Models (~10 lines)

In [ ]:
import ast
import numpy as np
import pandas as pd
import joblib

# Pick any model: ocean, hexaco, mbti, mmpi, scid, npi, dt4, tci, ...
SLUG = "ocean"

df = pd.read_csv(f"atlas/datasets/{SLUG}.csv")
embeddings = pd.read_csv(f"atlas/Embeddings/{SLUG}_embeddings.csv")
model = joblib.load(f"atlas/models/{SLUG}_rf_model.pkl")
encoder = joblib.load(f"atlas/models/{SLUG}_label_encoder.pkl")

X = np.array([ast.literal_eval(e) for e in embeddings["Embedding"]])
predictions = encoder.inverse_transform(model.predict(X))
accuracy = (predictions == df["Factor"].values).mean()

print(f"{SLUG.upper()}: {len(df)} traits, {len(set(predictions))} factors, accuracy = {accuracy:.1%}")
print(f"Factors: {sorted(set(predictions))}")

## 3. Atlas Overview — All 44 Models

In [ ]:
import os

slugs = sorted([f.replace(".csv", "") for f in os.listdir("atlas/datasets") if f.endswith(".csv")])
print(f"Atlas contains {len(slugs)} personality models:\n")

results = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    n_factors = df["Factor"].nunique()
    
    model = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    enc = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    emb = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X = np.array([ast.literal_eval(e) for e in emb["Embedding"]])
    preds = enc.inverse_transform(model.predict(X))
    acc = (preds == df["Factor"].values).mean()
    
    results.append({"Model": slug.upper(), "Traits": len(df), "Factors": n_factors, "Accuracy": f"{acc:.1%}"})

results_df = pd.DataFrame(results).sort_values("Accuracy", ascending=False)
results_df.index = range(1, len(results_df) + 1)
results_df

## 4. Reproduce PCA — Cross-Model Dimensionality Analysis (Paper Figure 5-8)

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

# Category assignments for the 7-category taxonomy
CATEGORIES = {
    "Trait-Based": ["ocean", "hex", "mbti", "epm", "sixteenpf", "ftm"],
    "Narcissism-Based": ["npi", "pni", "hsns", "dtm", "dt4", "ffni", "ffni_sf", "narq", "mcmin", "ipn"],
    "Motivational/Value": ["stbv", "sdt", "rft", "aam", "mst", "cs"],
    "Cognitive/Learning": ["pct", "cest", "scm", "fsls"],
    "Clinical/Health": ["mmpi", "scid", "bdi", "gad7", "wais", "tci", "mcmi", "tmp", "rit", "tat"],
    "Interpersonal/Conflict": ["disc", "tki"],
    "Application-Specific": ["riasec", "cmoa", "tei", "bt", "em", "papc"]
}
slug_to_cat = {s: c for c, slugs in CATEGORIES.items() for s in slugs}

# Load all embeddings
all_vecs, all_labels, all_cats = [], [], []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    emb = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X = np.array([ast.literal_eval(e) for e in emb["Embedding"]])
    all_vecs.append(X)
    all_labels.extend([slug.upper()] * len(X))
    all_cats.extend([slug_to_cat.get(slug, "Unknown")] * len(X))

X_all = np.vstack(all_vecs)
n_unknown = sum(1 for c in all_cats if c == "Unknown")
print(f"Loaded {X_all.shape[0]} embeddings ({X_all.shape[1]}-dim) from {len(slugs)} models")
if n_unknown > 0:
    unknown_slugs = sorted(set(s for s, c in zip([slugs[i] for i in range(len(slugs)) for _ in range(1)], all_cats) if c == "Unknown"))
    print(f"WARNING: {n_unknown} embeddings have Unknown category")

In [ ]:
# PCA — Scree plot (Paper Figure 5)
pca = PCA(n_components=50)
X_pca = pca.fit_transform(X_all)
cumvar = np.cumsum(pca.explained_variance_ratio_) * 100

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(range(1, 51), pca.explained_variance_ratio_ * 100, alpha=0.6, label="Individual")
ax.plot(range(1, 51), cumvar, "r-o", markersize=3, label="Cumulative")
ax.axhline(y=cumvar[-1], color="gray", linestyle="--", alpha=0.5)
ax.set_xlabel("Principal Component")
ax.set_ylabel("Variance Explained (%)")
ax.set_title(f"PCA Scree Plot — {X_all.shape[0]} Trait Embeddings from 44 Models\n50 PCs capture {cumvar[-1]:.1f}% of variance")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
# PCA — Model centroids in PC1-PC2 space (Paper Figure 6)
cat_colors = {
    "Trait-Based": "#1f77b4", "Narcissism-Based": "#ff7f0e",
    "Motivational/Value": "#2ca02c", "Cognitive/Learning": "#d62728",
    "Clinical/Health": "#9467bd", "Interpersonal/Conflict": "#8c564b",
    "Application-Specific": "#e377c2"
}

centroids = pd.DataFrame({
    "PC1": X_pca[:, 0], "PC2": X_pca[:, 1],
    "Model": all_labels, "Category": all_cats
}).groupby(["Model", "Category"])[["PC1", "PC2"]].mean().reset_index()

fig, ax = plt.subplots(figsize=(12, 8))
for cat, color in cat_colors.items():
    subset = centroids[centroids["Category"] == cat]
    ax.scatter(subset["PC1"], subset["PC2"], c=color, s=80, label=cat, alpha=0.8, edgecolors="white", linewidth=0.5)
    for _, row in subset.iterrows():
        ax.annotate(row["Model"], (row["PC1"], row["PC2"]), fontsize=7, alpha=0.7,
                    xytext=(4, 4), textcoords="offset points")

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.set_title("44 Personality Models in PC1-PC2 Space (Model Centroids)")
ax.legend(loc="best", fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# PCA — All traits colored by category (Paper Figure 8)
fig, ax = plt.subplots(figsize=(12, 8))
for cat, color in cat_colors.items():
    mask = [c == cat for c in all_cats]
    ax.scatter(X_pca[mask, 0], X_pca[mask, 1], c=color, s=4, alpha=0.3, label=cat)

ax.set_xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
ax.set_ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
ax.set_title(f"All {X_all.shape[0]} Trait Embeddings — 44 Models, 7 Categories")
ax.legend(markerscale=5, fontsize=8)
plt.tight_layout()
plt.show()

## 5. Cross-Model Search with FAISS

In [ ]:
import faiss

# Build FAISS index over entire atlas
X_norm = X_all / np.linalg.norm(X_all, axis=1, keepdims=True)
index = faiss.IndexFlatIP(X_norm.shape[1])
index.add(X_norm.astype(np.float32))

# Build metadata for lookups
all_factors = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    all_factors.extend(df["Factor"].values)

print(f"FAISS index: {index.ntotal} vectors, {X_norm.shape[1]}-dim")
print(f"Ready for cross-model personality search")

In [ ]:
# Query: find similar traits across all 44 models
# This demonstrates cross-category retrieval — the atlas's core value
query_slug = "ocean"
query_factor = "Extraversion"

query_df = pd.read_csv(f"atlas/datasets/{query_slug}.csv")
query_emb = pd.read_csv(f"atlas/Embeddings/{query_slug}_embeddings.csv")

# Find first row matching the target factor
idx = query_df[query_df["Factor"] == query_factor].index[0]
q = np.array([ast.literal_eval(query_emb["Embedding"].iloc[idx])]).astype(np.float32)
q = q / np.linalg.norm(q)

D, I = index.search(q, 20)

query_trait = query_df.iloc[idx]
print(f"Query: {query_slug.upper()} / {query_factor} — \"{query_trait['Adjective']}\"\n")
print(f"{'Rank':<5} {'Model':<12} {'Factor':<35} {'Category':<22} {'Score':.5}")
print("-" * 85)
for rank, (i, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"{rank:<5} {all_labels[i]:<12} {all_factors[i]:<35} {all_cats[i]:<22} {score:.4f}")

In [ ]:
# Second query: Dark Triad Machiavellianism → cross-category retrieval
# Shows how a narcissism-based construct connects to clinical, trait, and motivational models
query_slug = "dtm"
query_factor = "Machiavellianism"

query_df = pd.read_csv(f"atlas/datasets/{query_slug}.csv")
query_emb = pd.read_csv(f"atlas/Embeddings/{query_slug}_embeddings.csv")

idx = query_df[query_df["Factor"] == query_factor].index[0]
q = np.array([ast.literal_eval(query_emb["Embedding"].iloc[idx])]).astype(np.float32)
q = q / np.linalg.norm(q)

D, I = index.search(q, 20)

query_trait = query_df.iloc[idx]
print(f"Query: {query_slug.upper()} / {query_factor} — \"{query_trait['Adjective']}\"\n")
print(f"{'Rank':<5} {'Model':<12} {'Factor':<35} {'Category':<22} {'Score':.5}")
print("-" * 85)
for rank, (i, score) in enumerate(zip(I[0], D[0]), 1):
    print(f"{rank:<5} {all_labels[i]:<12} {all_factors[i]:<35} {all_cats[i]:<22} {score:.4f}")

# Count unique categories and models in results
result_cats = set(all_cats[i] for i in I[0])
result_models = set(all_labels[i] for i in I[0])
print(f"\n→ {len(result_cats)} categories, {len(result_models)} models in top 20 — cross-tradition retrieval")

## 6. Lexical Schema — What's in Each Model

In [ ]:
# Inspect any model's lexical schema
INSPECT = "hex"  # change to any slug (ocean, hex, mbti, mmpi, scid, npi, dt4, tci, ...)

df = pd.read_csv(f"atlas/datasets/{INSPECT}.csv")
print(f"{INSPECT.upper()}: {len(df)} traits across {df['Factor'].nunique()} factors\n")

for factor, group in df.groupby("Factor"):
    unique_adj = group["Adjective"].unique()[:5]
    print(f"  {factor} ({len(group)} traits): {', '.join(unique_adj)}, ...")

## 7. DSM-5 Clinical Alignment — Does the Atlas Match the Diagnostic Gold Standard?

222 DSM-5-TR disorders are embedded and queried against the atlas. If the atlas taxonomy is well-structured, clinical disorders should route to the **Clinical/Health** category.

---

**Repository:** [github.com/Wildertrek/survey](https://github.com/Wildertrek/survey)  
**Paper:** Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST.  
**License:** MIT

In [ ]:
import json
from collections import Counter

# Load pre-computed DSM-5 embeddings (no API key needed)
dsm5 = json.load(open("atlas/data/dsm5_disorders.json"))
dsm5_emb = pd.read_csv("atlas/data/dsm5_embeddings.csv")
dsm5_vecs = np.array([ast.literal_eval(e) for e in dsm5_emb["Embedding"]]).astype(np.float32)
dsm5_vecs = dsm5_vecs / np.linalg.norm(dsm5_vecs, axis=1, keepdims=True)

print(f"Loaded {len(dsm5)} DSM-5-TR disorders across {len(set(d['dsm5_category'] for d in dsm5))} categories")

# Query each disorder against the atlas FAISS index (built in Section 5)
k = 20
D_dsm, I_dsm = index.search(dsm5_vecs, k)

# For each disorder, what atlas category dominates the top-k results?
clinical_count = 0
dsm5_results = []
for idx, disorder in enumerate(dsm5):
    cats_found = Counter(all_cats[i] for i in I_dsm[idx] if i >= 0)
    top_cat = cats_found.most_common(1)[0][0]
    models_found = Counter(all_labels[i] for i in I_dsm[idx] if i >= 0)
    top_model = models_found.most_common(1)[0][0]
    is_clinical = top_cat == "Clinical/Health"
    if is_clinical:
        clinical_count += 1
    dsm5_results.append({
        "category": disorder["dsm5_category"],
        "disorder": disorder["disorder_name"],
        "top_atlas_cat": top_cat,
        "top_atlas_model": top_model,
        "clinical": is_clinical,
    })

rate = clinical_count / len(dsm5)
print(f"\nClinical alignment: {clinical_count}/{len(dsm5)} disorders route to Clinical/Health = {rate:.1%}")

# Which 4 disorders didn't route to Clinical?
non_clinical = [r for r in dsm5_results if not r["clinical"]]
if non_clinical:
    print(f"\nNon-clinical routing ({len(non_clinical)} disorders):")
    for r in non_clinical:
        print(f"  {r['disorder'][:60]:<62} → {r['top_atlas_cat']} ({r['top_atlas_model']})")

In [ ]:
# Per DSM-5 category breakdown: which atlas model dominates?
dsm5_cats = sorted(set(r["category"] for r in dsm5_results))
print(f"{'DSM-5 Category':<45} {'Disorders':>9} {'Clinical%':>10} {'Top Atlas Model':<15}")
print("-" * 85)

for cat in dsm5_cats:
    cat_rows = [r for r in dsm5_results if r["category"] == cat]
    n = len(cat_rows)
    clin = sum(1 for r in cat_rows if r["clinical"])
    models = Counter(r["top_atlas_model"] for r in cat_rows)
    top = models.most_common(1)[0][0]
    print(f"{cat[:44]:<45} {n:>9} {clin/n:>9.0%} {top:<15}")

print(f"\n{'TOTAL':<45} {len(dsm5_results):>9} {clinical_count/len(dsm5_results):>9.1%}")

## 8. Experiment 2 — 3072-dim Embedding Upgrade (Optional)

The atlas also includes upgraded **3072-dim** embeddings (`text-embedding-3-large`) and retrained RF classifiers from Experiment 2. These are hosted on [Hugging Face Hub](https://huggingface.co/datasets/Wildertrek/personality-atlas-3072) due to size (547 MB total).

Run the cells below to download and compare 1536 vs 3072 accuracy.

In [ ]:
!pip install -q huggingface_hub

from huggingface_hub import hf_hub_download

HF_REPO = "Wildertrek/personality-atlas-3072"

def load_3072(slug):
    """Download and load 3072-dim assets for a single model from HuggingFace."""
    emb_path = hf_hub_download(HF_REPO, f"Embeddings_3072/{slug}_embeddings.csv", repo_type="dataset")
    model_path = hf_hub_download(HF_REPO, f"models_3072/{slug}_rf_model.pkl", repo_type="dataset")
    enc_path = hf_hub_download(HF_REPO, f"models_3072/{slug}_label_encoder.pkl", repo_type="dataset")
    emb_df = pd.read_csv(emb_path)
    X = np.array([ast.literal_eval(e) for e in emb_df["Embedding"]])
    return X, joblib.load(model_path), joblib.load(enc_path)

print("Ready to download 3072-dim assets from HuggingFace.")

In [ ]:
# Compare 1536 vs 3072 accuracy across all 44 models
comparison = []
for slug in slugs:
    df = pd.read_csv(f"atlas/datasets/{slug}.csv")
    y_true = df["Factor"].values

    # 1536-dim (already loaded from repo)
    m1536 = joblib.load(f"atlas/models/{slug}_rf_model.pkl")
    e1536 = joblib.load(f"atlas/models/{slug}_label_encoder.pkl")
    emb1536 = pd.read_csv(f"atlas/Embeddings/{slug}_embeddings.csv")
    X1536 = np.array([ast.literal_eval(e) for e in emb1536["Embedding"]])
    acc1536 = (e1536.inverse_transform(m1536.predict(X1536)) == y_true).mean()

    # 3072-dim (downloaded from HuggingFace)
    X3072, m3072, e3072 = load_3072(slug)
    acc3072 = (e3072.inverse_transform(m3072.predict(X3072)) == y_true).mean()

    comparison.append({
        "Model": slug.upper(),
        "1536-dim": f"{acc1536:.1%}",
        "3072-dim": f"{acc3072:.1%}",
        "Delta": f"{(acc3072 - acc1536)*100:+.1f}pp",
    })

comp_df = pd.DataFrame(comparison).sort_values("Delta", ascending=False)
comp_df.index = range(1, len(comp_df) + 1)

improved = sum(1 for c in comparison if float(c["3072-dim"].rstrip("%")) > float(c["1536-dim"].rstrip("%")))
mean_1536 = np.mean([float(c["1536-dim"].rstrip("%")) for c in comparison])
mean_3072 = np.mean([float(c["3072-dim"].rstrip("%")) for c in comparison])

print(f"1536-dim mean: {mean_1536:.1f}% | 3072-dim mean: {mean_3072:.1f}% | Delta: {mean_3072 - mean_1536:+.1f}pp")
print(f"{improved}/44 models improved with 3072-dim embeddings\n")
comp_df

---

**Repository:** [github.com/Wildertrek/survey](https://github.com/Wildertrek/survey) | **3072-dim assets:** [Hugging Face Hub](https://huggingface.co/datasets/Wildertrek/personality-atlas-3072)  
**Paper:** Raetano, J., Gregor, J., & Tamang, S. (2026). *A Survey and Computational Atlas of Personality Models.* ACM TIST.  
**License:** MIT